# **Tulbad:**


*   Kollased - välja võtta
*   Sinised - lahku lüüa "/"
*   Beež - küsitav







In [1]:
import numpy as np
import pandas as pd
from copy import *
import numbers
# TODO
data = pd.read_csv("clinvar_conflicting.csv")
data1 = deepcopy(data)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (0,38,40) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
#muudame nende tulpade sisu
data1[["Amino_acids algne", "Amino_acids asendus"]] = data1.Amino_acids.str.split("/",expand=True)
data1[["Codons algne", "Codons asendus"]] = data1.Codons.str.split("/",expand=True)
#eemaldame mittevajalikud tulbad
välja = ["CLNDNINCL", "CLNSIGINCL", "Allele", "Amino_acids", "Codons", "CLNDN", "CLNDISDB", "CLNDISDBINCL", "CLNHGVS", "CLNVI", "MC", "SSR", "Feature_type", "Feature", "DISTANCE", "BAM_EDIT", "MOTIF_NAME", "MOTIF_POS", "HIGH_INF_POS", "MOTIF_SCORE_CHANGE"]
for i in välja:
	data1 = data1.drop(labels=[i], axis=1)

#ref tulp kõik mis on rohkem kui 1 täht
data1.head()

#LIHTSASTI DEFINEERITAVAD TULBAD
inttüüpi = ["POS", "ORIGIN", "CLASS"]
floattüüpi = ["AF_ESP", "AF_EXAC", "AF_TGP", "LoFtool", "CADD_PHRED", "CADD_RAW" ,  "STRAND", "BLOSUM62" ]
strtüüpi = ["REF", "ALT", "Consequence", "IMPACT", "SYMBOL", "BIOTYPE" , "CLNVC", "IMPACT", "SIFT", "PolyPhen", "Amino_acids algne", "Amino_acids asendus", "Codons algne", "Codons asendus", "cDNA_position" , "CDS_position", "Protein_position"]


def datatypes(inttüüpi, strtüüpi, floattüüpi):
  for i in range(len(inttüüpi)):
      data1[inttüüpi[i]]=data1[inttüüpi[i]].astype(int)

  for i in range(len(strtüüpi)):
      data1[strtüüpi[i]]=data1[strtüüpi[i]].astype(str)
  
  for i in range(len(floattüüpi)):
      data1[floattüüpi[i]]=data1[floattüüpi[i]].astype(float)
      
datatypes(inttüüpi, strtüüpi, floattüüpi)
#ISIKLIKKU LÄHENEMIST VAJAVAD TULBAD
##CHROM
data1["CHROM"]=data1["CHROM"].astype(str)
data1.loc[data1["CHROM"] == "X", "CHROM"] = "23"
data1.loc[data1["CHROM"] == "MT", "CHROM"] = "24"

In [3]:
pd.set_option('display.max_columns', None)
data1["EXON"]=data1["EXON"].astype(str)
data1["INTRON"]=data1["INTRON"].astype(str)

data1.loc[(data1["EXON"]!='nan') & (data1["INTRON"]!='nan') & (data1["REF"].str.len() == 1), ["INTRON", "EXON"]] = "TÕRGE"
data1.loc[data1["EXON"] == data1["INTRON"], ["INTRON", "EXON"]] = "TÕRGE"


print(data1[data1["INTRON"] == "TÕRGE"])

data1["EXON"].str.contains('oct').tolist().count(True)
#vaja on, et NaN'idest tekiksid eraldi tulbad mille saaks droppida
#vaja on midagi exonite ja intronitega teha: kui on variant eksonism ei saa ta intronis enam olla ja vastupidi



#CDNA, CDS ja protein tulpade töötlus
data1[["cDNA_pos_start" , "cDNA_pos_end"]] = data1.cDNA_position.str.split("-",expand=True)
data1[["CDS_pos_start" , "CDS_pos_end"]] = data1.CDS_position.str.split("-",expand=True)
data1[["Protein_pos_start" , "Protein_pos_end"]] = data1.Protein_position.str.split("-",expand=True)

data1.loc[data1["cDNA_pos_end"] == 'nan', "cDNA_pos_end"] = data1["cDNA_pos_start"]
data1.loc[data1["CDS_pos_end"] == 'nan', "CDS_pos_end"] = data1["CDS_pos_start"]
data1.loc[data1["Protein_pos_end"] == 'nan', "Protein_pos_end"] = data1["Protein_pos_start"]

data1.loc[data1["cDNA_position"].str.contains("?"), ["cDNA_position"]] = np.nan

      CHROM       POS REF ALT  AF_ESP  AF_EXAC  AF_TGP  \
5227     19   1397399   C   T  0.0002  0.00028  0.0000   
5228     19   1397443   G   A  0.0712  0.07554  0.0365   
5473     19  35530029   G   A  0.0002  0.00007  0.0000   
5474     19  35530580   G   A  0.0003  0.00050  0.0004   
6940      1  40539727   C   T  0.0000  0.00016  0.0000   
...     ...       ...  ..  ..     ...      ...     ...   
63480    24     15470   T   C  0.0000  0.00000  0.0000   
63481    24     15511   T   C  0.0000  0.00000  0.0000   
63482    24     15923   A   G  0.0000  0.00000  0.0000   
64248    23  55057617   G   C  0.0000  0.00000  0.0027   
64358    23  73641251   A   T  0.0009  0.00144  0.0000   

                           CLNVC  ORIGIN  CLASS              Consequence  \
5227   single_nucleotide_variant       1      1  downstream_gene_variant   
5228   single_nucleotide_variant       1      0  downstream_gene_variant   
5473   single_nucleotide_variant       1      1    upstream_gene_variant   

In [4]:
data1 = pd.get_dummies(data1,columns=list(data1.select_dtypes('object')))
